In [1]:
import requests
import pandas as pd
import os
#Also requires the lxml module

In [2]:
r_lakes = requests.get('https://coastwatch.glerl.noaa.gov/statistic/physical.html')
t_lakes = r_lakes.text
df_lakes = pd.read_html(t_lakes)[1]

In [3]:
df_lakes[0][0] = 'Measurement'
df_lakes[1][0] = 'Unit'
df_lakes.columns = df_lakes.iloc[0]
df_lakes = df_lakes.drop(df_lakes.index[0])

In [4]:
#remove trailing (a,b,c), copy down 1 column (0), merge columns (0,1), remove totals?, pivot, make lakes index
prev = None
temp = df_lakes['Measurement'].dropna()
for i in range(2,len(df_lakes['Measurement']),2):
    if df_lakes['Measurement'][i]:
        df_lakes['Measurement'][i] = df_lakes['Measurement'][i-1]
df_lakes = df_lakes.set_index(['Measurement','Unit'])
df_lakes

0                                        Superior             Michigan  \
Measurement         Unit                                                 
Elevationa          (feet)                    600                  577   
                    (metres)                  183                  176   
Length              (miles)                   350                  307   
                    (kilometres)              563                  494   
Breadth             (miles)                   160                  118   
                    (kilometres)              257                  190   
Average Deptha      (feet)                    483                  279   
                    (metres)                  147                   85   
Maximum Deptha      (feet)                   1332                  925   
                    (metres)                  406                  282   
Volumea             (cu. miles)              2900                 1180   
                    (km3)                   12100                 4920   
Water Area          (sq. mi.)               31700                22300   
                    (km2)                   82100                57800   
Land Drainage Areab (sq. mi.)               49300                45600   
                    (km2)                  127700               118000   
Total Area          (sq. mi.)               81000                67900   
                    (km2)                  209800               175800   
Shoreline Lengthc   (miles)                  2726                 1638   
                    (kilometres)             4385                 2633   
Retention Time      (years)                   191                   99   
Outlet              Outlet        St. Marys River  Straits of Mackinac   

0                                           Huron  \
Measurement         Unit                            
Elevationa          (feet)                    577   
                    (metres)                  176   
Length              (miles)                   206   
                    (kilometres)              332   
Breadth             (miles)                   183   
                    (kilometres)              245   
Average Deptha      (feet)                    195   
                    (metres)                   59   
Maximum Deptha      (feet)                    750   
                    (metres)                  229   
Volumea             (cu. miles)               850   
                    (km3)                    3540   
Water Area          (sq. mi.)               23000   
                    (km2)                   59600   
Land Drainage Areab (sq. mi.)               51700   
                    (km2)                  134100   
Total Area          (sq. mi.)               74700   
                    (km2)                  193700   
Shoreline Lengthc   (miles)                  3827   
                    (kilometres)             6157   
Retention Time      (years)                    22   
Outlet              Outlet        St. Clair River   

0                                                         Erie  \
Measurement         Unit                                         
Elevationa          (feet)                                 569   
                    (metres)                               173   
Length              (miles)                                241   
                    (kilometres)                           388   
Breadth             (miles)                                 57   
                    (kilometres)                            92   
Average Deptha      (feet)                                  62   
                    (metres)                                19   
Maximum Deptha      (feet)                                 210   
                    (metres)                                64   
Volumea             (cu. miles)                            116   
                    (km3)                                  484   
Water Area          (sq. m

In [5]:
metric_lakes = df_lakes.drop(['(feet)','(miles)','(cu. miles)','(sq. mi.)'],level='Unit').drop(axis=1,columns=['Totals'])
metric_lakes = metric_lakes.transpose()
metric_lakes.columns = [' '.join(col).strip()\
                        .replace('metres','m').replace('kilo','k').replace('years','yr')
                        for col in metric_lakes.columns.values]
metric_lakes

,Elevationa (m),Length (km),Breadth (km),Average Deptha (m),Maximum Deptha (m),Volumea (km3),Water Area (km2),Land Drainage Areab (km2),Total Area (km2),Shoreline Lengthc (km),Retention Time (yr),Outlet Outlet
0,,,,,,,,,,,,
Superior,183,563,257,147,406,12100,82100,127700,209800,4385,191,St. Marys River
Michigan,176,494,190,85,282,4920,57800,118000,175800,2633,99,Straits of Mackinac
Huron,176,332,245,59,229,3540,59600,134100,193700,6157,22,St. Clair River
Erie,173,388,92,19,64,484,25700,78000,103700,1402,2.6,Niagara River/ Welland Canal
Ontario,74,311,85,86,244,1640,18960,64030,82990,1146,6,St. Lawrence River


In [6]:
metric_lakes.to_csv(os.path.join('..','data_files','clean_data','lake_stats.csv'))